<a href="https://colab.research.google.com/github/zebedy-magic/BottleImageRecognition/blob/master/Bottle_recognition_part4_ipynb_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lets try a neural network and 
Lets experiment with our images 
 
 

*   I am going to create a really simple network first to see if a binary classiifcation works
*   Next maybe a CNN convoluted nueral network for more sophisticated recognition



In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pandas as pd
from os.path import join
from PIL import Image
import numpy as np

#root directory
root='/content/drive/My Drive/'

image_path=join(root,'images','processed')



Using TensorFlow backend.


In [2]:
def get_image(filename,image_path):
  #get the image
  img=Image.open(join(image_path,filename))
  return np.array(img)

In [3]:
def create_matrix(labels):
  #create feature matrix using the creat features 
  lst_images=[]
  #loop through the images .get the features and append to a list
  for image in labels.index:
    img_arr=get_image(image,image_path)
    img_flat=img_arr.flatten()
   #use the standard _scaler on each image -grey scale so only one channel  
    #ss=StandardScaler()
    #ss_matrix=ss.fit_transform(img_flat)
    
    lst_images.append(img_flat)
  #use array to make it into one big matrix
  matrix=np.array(lst_images)
  return matrix

In [46]:
#load the training set 
labels=pd.read_csv(join(root,'labels.csv'),index_col=0)
print(labels.head())
print(labels.info())
target=to_categorical(labels['bottle'])


                                 bottle
image                                  
2020-04-25 12:01:24.739043.jpeg       1
2020-04-25 12:01:26.103276.jpeg       0
2020-04-25 12:01:27.977304.jpeg       0
2020-04-25 12:01:29.199803.jpeg       1
2020-04-25 12:01:30.760437.jpeg       0
<class 'pandas.core.frame.DataFrame'>
Index: 439 entries, 2020-04-25 12:01:24.739043.jpeg to 2020-06-23 08:54:16.864046.jpeg
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   bottle  439 non-null    int64
dtypes: int64(1)
memory usage: 6.9+ KB
None


In [47]:


matrix=create_matrix(labels)
print(matrix.shape)

ss=StandardScaler()
ss_matrix=ss.fit_transform(matrix)



(439, 40000)


In [55]:
#lets create a basic dense network with keras and use the processed images 

early_stopping_monitor=EarlyStopping(patience=3)

model= Sequential()

model.add(Dense(200,activation='relu',input_shape=(40000,)))

model.add(Dense(200,activation='relu'))



model.add(Dense(2,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(ss_matrix,target,validation_split=0.2,epochs=30,callbacks=[early_stopping_monitor])

Train on 351 samples, validate on 88 samples
Epoch 1/30
351/351 [==============================] - 1s 4ms/step - loss: 9.4744 - accuracy: 0.7379 - val_loss: 1.0623 - val_accuracy: 0.9432
Epoch 2/30
351/351 [==============================] - 1s 4ms/step - loss: 6.5820 - accuracy: 0.8319 - val_loss: 0.8913 - val_accuracy: 0.9773
Epoch 3/30
351/351 [==============================] - 1s 3ms/step - loss: 1.8686 - accuracy: 0.9231 - val_loss: 0.0408 - val_accuracy: 0.9886
Epoch 4/30
351/351 [==============================] - 1s 3ms/step - loss: 0.8655 - accuracy: 0.9544 - val_loss: 0.3390 - val_accuracy: 0.9773
Epoch 5/30
351/351 [==============================] - 1s 3ms/step - loss: 0.7981 - accuracy: 0.9516 - val_loss: 0.2475 - val_accuracy: 0.9773
Epoch 6/30
351/351 [==============================] - 1s 3ms/step - loss: 0.9784 - accuracy: 0.9288 - val_loss: 0.3527 - val_accuracy: 0.9773


In [52]:
#lets try and evaluate on unknown test set 
#load in test_data
labels_test=pd.read_csv(join(root,'test_labels.csv'),index_col=0)
print(labels_test.head())
print(labels_test.info())
target_test=to_categorical(labels_test['bottle'])



                                 bottle
image                                  
2020-06-23 08:52:18.638263.jpeg       0
2020-06-23 08:52:26.978667.jpeg       0
2020-06-23 08:52:28.502669.jpeg       0
2020-06-23 08:52:30.254279.jpeg       0
2020-06-23 08:52:32.002719.jpeg       0
<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 2020-06-23 08:52:18.638263.jpeg to 2020-06-23 08:52:44.876371.jpeg
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   bottle  10 non-null     int64
dtypes: int64(1)
memory usage: 160.0+ bytes
None


In [53]:


matrix_test=create_matrix(labels_test)
print(matrix_test.shape)


ss_matrix_test=ss.fit_transform(matrix_test)



(10, 40000)


In [56]:
model.evaluate(ss_matrix_test,target_test)

10/10 [==============================] - 0s 1ms/step


[2.1252803802490234, 0.8999999761581421]

lets use a CNN and see if this is better 
